# Notes on Manabe/Wetherald 1967
## Thermal Equilibrium of the Atmosphere with a Given Distribution of Relative Humidity


## 1. Introduction

- It's a follow up study
- Preceded by papers:
   - [1964 Manabe/Strickler](https://www.gfdl.noaa.gov/bibliography/related_files/sm6401.pdf): thermal equilibrium of the atmosphere with convective adjustment
      - Humidity correction factor was a given vertical distribution of absolute humidity, not relative, not as a function of temperature
   - [1961 Manabe & Möller](https://journals.ametsoc.org/view/journals/mwre/89/12/1520-0493_1961_089_0503_otreah_2_0_co_2.xml): radiative convective equilibrium
      - mentions this [1956 paper by Phillips](https://empslocal.ex.ac.uk/people/staff/gv219/classics.d/Phillips56.pdf) modeling energy flows numerically assuming a heat source near the equator and heat sink near the poles.
   - [1965 Manabe et al](https://journals.ametsoc.org/view/journals/mwre/93/12/1520-0493_1965_093_0769_scoagc_2_3_co_2.xml): Use computation (numerical integration) to model radiative transfer in the atmosphere including the hydrologic cycle
      - Used a climatological distribution of absolute humidity as approximation
    
- Why change the model from absolute to RH?
  - Absolute humidity is highly coupled to temperature
  - Average RH latitude-height distributions appear similar in winter and summer, whereas AH equivalents do not
    - They don't provide a plot for the latter
  - Maybe you can assume a given (fixed) vertical distribution of RH since it doesn't change much seasonally
  - Removing an unknown saves effort for parameters that are dynamic & saves computation cycles
  - If RH distribution is static, absolute humidity is a simple function of temperature (for a fixed altitude+latitude)
  - If atmospheric temperatures are higher, then the absolute humidity at any given altitude is higher
  - Long-wave infrared (LWIR) emissions to space mostly happen at the altitude surface where there isn't much more LWIR-absorbent substance (e.g. humidity) at a higher altitude to absorb outgoing photons and re-emit them downward
    - Higher absolute humidity at all altitudes -> effective altitude of outbound LWIR emission gets higher
      - Higher altitude => larger surface area of LWIR emission, A = 4 * pi * r^2
      - _If the surface area grows but the outgoing LWIR power is fixed, power density (W/m2) decreases_
    - What are the effects on the convective equilibrium if this occurs?
      
- Next step (this paper): numerically solve the fully coupled radiative transfer and hydrological cycle elements.
- But before doing so, compute a few radiative convective equilibria with fixed relative humidity to answer:
  1) How fast does the atmosphere equilibriate when RH is realistic but time-invariant?
  2) How do optical/radiative variables affect the atmosphere's equilibrium temperature under realistic but static RH conditions?
  3) From the atmosphere temperature calculated in 1) and 2), what's the surface equilibrium temp?
- Use those answers to sanity check more complex models

- Möller 1963 discussed CO2 content vs. magnitude of LWIR at earth's surface relating to equilibrium temperature
  - Relative vs. absolute humidity confusion
  - Similar order-of-magnitude dependence of equilibrium temperature on CO2 content vs papers from the past 15 years
  - "We should probably re-evaluate"


### Quick terminology check
What is relative vs absolute humidity?
- Absolute aka specific humidity: grams H2O vapor per m^3, not including water droplets
- Relative humidity (RH): density of vapor (not suspended droplets) in the air as a percentage of the saturation vapor pressure, determined by Clausius-Clapeyron equation (or similar)

## 2. Radiative convective equilibrium

This is a long section, stretches across 5 pages.

Setting up the T&Cs of the model at equilibrium:

  - net incoming solar and outgoing LWIR energy are equal where the atmosphere ends 
  - No temperature discontinuities are allowed
  - Critical lapse rate: 6.5C/km, above it convection/eddies will crush it back to 6.5C/km, below it radiative equilibrium is satisfied
  - Heat capacity of Earth's surface = 0 (model is simplified, ignores oceans)
  - New for this paper: RH in the atmosphere is some given vertical distribution (maybe compiled from radiosonde data?)

Some notes
- Radiation balance defines when we reach equilibrium, otherwise temperature would change
- What is lapse rate? Change in atmospheric temperature with altitude
- #2: discontinuities imply either free energy that should dissipate, not good for a numerical simulation, or a solid insulating barrier which doesn't exist


Numerical formulation is as an initial value problem that they let asymptote to some equilibrium (see Appendix 1)
- "Marching computation": Figure 2
  - compute mixing ratio of water vapor r(time), (why does the water vapor mixing ratio matter?)
    - `r = 0.622*h*e_s(T) / (p - h*e_s(T))`
  - compute mean emissivity and absorptivity,
  - compute temperature change due to blackbody radiation,
  - timestep: compute next temperature from last temperature + temperature change from BB radiation,
  - update convective effects (?),
  - loop
- e_s(time) is saturation vapor pressure of water vapor as a function of temperature, h = relative humidity
- I is the "indexing of the finite differences in the vertical direction"
  - Seems like vertical axis is modeled in discrete altitude blocks

In [2]:
# Plotting the mixing ratio as a function of e_s
# %pip install matplotlib mlx
import matplotlib.pyplot as plt
import mlx.core as mx
p = 1000 # assume some static pressure
e_s = mx.arange(100) + 1 # saturation pressure is some value between 1-101 (arb units)
h = (mx.arange(100)/100) # relative humidity is 0.0 to 1.0
r = mx.outer(0.622*e_s / (p - h*e_s), h)

#fig, ax = plt.subplots()
#plt.imshow(r)
#cb = plt.colorbar()
#plt.xlabel("h?")
#plt.ylabel("e_s?")
# r = 0.622*h*e_s(T) / (p - h*e_s(T))


## 2. Radiative convective equilibrium (continued)
- Simplification from absolute -> relative humidity means absolute humidity is essentially directly computable from temperature if the RH distribution is fixed and known.
- Heat capacity of moist air is needed to convert RH to AH
  - `C'_p = C_p[1 + (L/C_p)*pd_wrt_T( (0.622*he_s(T)) / (p - he_s(T)) )]`
  - L is latent heat of evaporation
  - C_p is specific heat of air under constant pressure
  - Term in brackets after `1 +` is due to 'change of latent energy of the air'
- Other computations are copied directly from Manabe/Strickler 1964
  - LWIR flux, solar radiation depletion (?), mean absorptivity and emissivity
